## Import Libraries

In [ ]:
import os
import shutil

import tensorflow as tf
keras = tf.keras

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

## Preparing Data

In [ ]:
os.chdir('drive/My Drive/AI Project/RPS-Intelligence')

In [ ]:
base_dir = 'rockpaperscissors'
IMG_SIZE = 150

### Move Images To Each Class Folders

In [ ]:
# Membagi gambar menjadi train set (60%) dan validation set (40%)
def move_images(valid_size = 0.4):
    all_class = {'train': {},'valid': {}}
    for cl in os.listdir(base_dir):
        items = os.listdir(f'{base_dir}/{cl}')
        valid_count = int(valid_size * (len(items)))
        all_class['train'][cl] = items[valid_count:]
        all_class['valid'][cl] = items[:valid_count]

    for category, cl in all_class.items():
        for sub_class, item in cl.items():
            for i in item:
                print(f'{category}/{sub_class}/{i}')
                des_path = f'{base_dir}/{category}/{sub_class}/'
                os.makedirs(des_path, exist_ok=True)
                shutil.move(f'{base_dir}/{sub_class}/{i}', des_path)

In [ ]:
move_images()

### Image Data Generator

In [ ]:
train_dir = os.path.join(base_dir, 'train')
valid_dir = os.path.join(base_dir, 'valid')

In [ ]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical'
)

In [ ]:
valid_generator = train_datagen.flow_from_directory(
    valid_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=32,
    class_mode='categorical'
)

## Building Neural Network

### Sequential Model

In [ ]:
model = keras.Sequential([
    # Input Layer
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    
    # Hidden Layer
    keras.layers.Conv2D(64, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Conv2D(128, kernel_size=(3,3), activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Dropout(0.2),
    keras.layers.Flatten(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.2),
    
    # Output Layer
    keras.layers.Dense(3),
    keras.layers.Activation('softmax')
])

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.RMSprop(lr=1e-4),
    metrics=['acc']
)

### Callback Function

In [ ]:
import datetime
logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)

### Training Model

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=40,
    epochs=50,
    validation_data=valid_generator,
    validation_steps=20,
    verbose=1,
    callbacks=[early_stopping, tensorboard_callback]
)

In [ ]:
model.save(datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '-RPS-Model.h5')

## Tensorboard For Model Evaluation

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

## Prediction Demonstration

In [ ]:
from google.colab import files
import matplotlib.image as mpimg
image = keras.preprocessing.image

In [ ]:
classes = {y:x for x,y in train_generator.class_indices.items()}

In [ ]:
uploaded = files.upload()
for fn in uploaded.keys():
    path = fn
    img = image.load_img(path, target_size=(IMG_SIZE, IMG_SIZE))
    imgplot = plt.imshow(img)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    prediction = model.predict(images, batch_size=10)
    
    print(fn)
    print(prediction)
    print(f'Prediction Class : {classes[np.argmax(prediction)]}')